In [1]:
import polars as pl
import plotly.express as px
import datetime as dt
import pandas as pd


df_base_peers_new = pl.read_excel("C:/Users/VHENEQUIM/Documents/Estudo/offshorePeer/base_peers_new_mapped.xlsx")
df_base_peers_old = pl.read_excel("C:/Users/VHENEQUIM/Documents/Estudo/offshorePeer/base_peers_old_mapped.xlsx")
df_base_peers_old = df_base_peers_old.filter(~pl.col("Mês").str.contains("06/2024|07/2024|08/2024|09/2024|10/2024|11/2024|12/2024"))

df_peers = pl.concat([df_base_peers_new, df_base_peers_old])
df_peers = df_peers.rename({
    "Fundo": "funds",
    "Mês": "month",
    "Pat. Liq.": "liquid_patrimony",
    "Ativo": "investment",
    "%Patrim. Liq.": "percentage",
    "R$Patrim. Liq.": "value"
}).with_columns(
    pl.col("month").str.to_datetime("%m/%Y").alias("date_for_sort")
).filter(
    (pl.col("investment_type") == "BDR") | (pl.col("investment_type") == "Invest. Ext.")
)
# Filter out months newer than 12/2024
cutoff_date = dt.datetime(2024, 12, 31)
df_peers = df_peers.filter(pl.col("date_for_sort") <= cutoff_date)

df_peers.write_excel("base_peers_mapped.xlsx")

In [2]:
df_peers.columns

['funds',
 'month',
 'liquid_patrimony',
 'investment',
 'percentage',
 'value',
 'ticker',
 'company_country',
 'area_of_work',
 'investment_type',
 'date_for_sort']

In [3]:
def graph_per_ticker(df, investment_type, value_type = "value"):
    if investment_type == "BDR":
        df = df.filter(pl.col("investment_type") == "BDR")
    elif investment_type == "Invest. Ext.":
        df = df.filter(pl.col("investment_type") == "Invest. Ext.")
    elif investment_type == None:
        investment_type = "BDR & Invest. Ext."
    
    # Group by month and ticker, sum the values
    monthly_ticker_sum = df.group_by(["month", "ticker", "date_for_sort"]).agg(
        pl.sum(value_type).alias("total_value")
    )

    # Convert to pandas for easier manipulation with plotly
    plot_df = monthly_ticker_sum.to_pandas()

    # Sort by date to ensure chronological order
    plot_df = plot_df.sort_values('date_for_sort')
    
    # Calculate the total value per ticker across all months
    ticker_totals = plot_df.groupby('ticker')['total_value'].sum().sort_values(ascending=False)
    
    # Get ordered list of tickers for the color assignment
    ordered_tickers = ticker_totals.index.tolist()
    
    # Create stacked bar chart with plotly using the custom order for color
    fig = px.bar(
        plot_df,
        x="month",
        y="total_value",
        color="ticker",
        title=f'Investment Values by Ticker and Month - {investment_type}',
        labels={
            "month": "Month",
            "total_value": "Total Value (R$)",
            "ticker": "Ticker"
        },
        category_orders={
            "month": plot_df["month"].tolist(),  # Preserves chronological order
            "ticker": ordered_tickers,  # Use the custom order based on total values
        },
        height=600,
    )

    # Improve layout
    fig.update_layout(
        xaxis_title="Month",
        yaxis_title="Total Value (R$)",
        legend_title="Ticker",
        barmode='relative',  # Use relative instead of stack to properly handle negative values
        hovermode="closest"
    )

    # Format hover info
    fig.update_traces(
        hovertemplate="<b>%{x}</b><br>" +
                    "Ticker: %{fullData.name}<br>" +
                    "Value: R$ %{y:,.2f}<br>"
    )

    # Show the plot
    fig.show()

    return None

graph_per_ticker(df_peers, "BDR")
graph_per_ticker(df_peers, "BDR", "percentage")
graph_per_ticker(df_peers, "Invest. Ext.")
graph_per_ticker(df_peers, "Invest. Ext.", "percentage")
graph_per_ticker(df_peers, None)
graph_per_ticker(df_peers, None, "percentage")


In [4]:
def graph_per_fund(df, fund_name, value_type="value", investment_type=None):
    """
    Generate a graph showing the investment composition of a specific fund over time.
    
    Parameters:
    -----------
    df : polars.DataFrame
        The dataframe containing the investment data
    fund_name : str
        The name of the fund to analyze
    value_type : str, default="value"
        The column to use for values ("value" for absolute or "percentage" for percentage)
    investment_type : str or None, default=None
        Filter for specific investment type ("BDR", "Invest. Ext.", or None for both)
    """
    # Filter for the specified fund
    filtered_df = df.filter(pl.col("funds") == fund_name)
    
    # Apply investment type filter if specified
    if investment_type == "BDR":
        filtered_df = filtered_df.filter(pl.col("investment_type") == "BDR")
        type_title = "BDR"
    elif investment_type == "Invest. Ext.":
        filtered_df = filtered_df.filter(pl.col("investment_type") == "Invest. Ext.")
        type_title = "Invest. Ext."
    else:
        type_title = "BDR & Invest. Ext."
    
    # Set value type label for the title
    value_label = "Percentage" if value_type == "percentage" else "Value (R$)"
    
    # Group by month, ticker, and date_for_sort, sum the values
    monthly_ticker_sum = filtered_df.group_by(["month", "ticker", "date_for_sort"]).agg(
        pl.sum(value_type).alias("total_value")
    )
    
    # Convert to pandas for easier manipulation with plotly
    plot_df = monthly_ticker_sum.to_pandas()
    
    # Check if data exists
    if plot_df.empty:
        print(f"No data found for fund: {fund_name} with investment type: {type_title}")
        return None
    
    # Sort by date to ensure chronological order
    plot_df = plot_df.sort_values('date_for_sort')
    
    # Calculate the total value per ticker across all months
    ticker_totals = plot_df.groupby('ticker')['total_value'].sum().sort_values(ascending=False)
    
    # Get ordered list of tickers for the color assignment
    ordered_tickers = ticker_totals.index.tolist()
    
    # Create stacked bar chart with plotly
    fig = px.bar(
        plot_df,
        x="month",
        y="total_value",
        color="ticker",
        title=f'{fund_name} - {type_title} ({value_label})',
        labels={
            "month": "Month",
            "total_value": value_label,
            "ticker": "Ticker"
        },
        category_orders={
            "month": plot_df["month"].tolist(),  # Preserves chronological order
            "ticker": ordered_tickers,  # Use the custom order based on total values
        },
        height=600,
    )
    
    # Improve layout
    fig.update_layout(
        xaxis_title="Month",
        yaxis_title=value_label,
        legend_title="Ticker",
        barmode='relative',  # Use relative instead of stack to properly handle negative values
        hovermode="closest"
    )
    
    # Format hover info
    hover_format = "%{y:.2f}%" if value_type == "percentage" else "R$ %{y:,.2f}"
    fig.update_traces(
        hovertemplate="<b>%{x}</b><br>" +
                      "Ticker: %{fullData.name}<br>" +
                      f"Value: {hover_format}<br>"
    )
    
    # Show the plot
    fig.show()
    
    return None

for fund in df_peers["funds"].unique():
    graph_per_fund(df_peers, fund)

# Example usage
# graph_per_fund(df_peers, "VINLAND")
# graph_per_fund(df_peers, "Fund Name Here", "percentage")
# graph_per_fund(df_peers, "Fund Name Here", investment_type="BDR")